In [1]:
import os, sys
import numpy as np
import pandas as pd
import EMdata
import time

In [24]:
#data path
file_path='F:/script/class2vec/real_star_file/10943_subset1_major.star'
datatype=0 #0 is relion 3.1, 1 is relion 3, 2 is cryosparc
fast=True
block_size=64
file_name=os.path.basename(file_path)
output_path=os.path.dirname(file_path)+'/'+os.path.splitext(file_name)[0]
#if os.path.isdir(output_path) is False:
#    os.mkdir(output_path)

In [25]:
import gemmi
#copy from https://github.com/jianglab/HILL/blob/main/hill_power_spectra.py
def star2dataframe(starFile):
    import pandas as pd
    from gemmi import cif
    star = cif.read_file(starFile)
    if len(star) == 2:
        optics = cif.Document()
        optics.add_copied_block(star[0])
        del star[0]
        js = optics.as_json(True)  # True -> preserve case
        optics = pd.read_json(js).T
        d = {c.strip('_'): optics[c].values[0] for c in optics}
        optics = pd.DataFrame(d)
    else:
        optics = None
    js = star.as_json(True)  # True -> preserve case
    data = pd.read_json(js).T
    d = {c.strip('_'): data[c].values[0] for c in data}
    data = pd.DataFrame(d)
    
    assert("rlnImageName" in data)
    tmp = data["rlnImageName"].str.split("@", expand=True)
    indices, filenames = tmp.iloc[:,0], tmp.iloc[:, -1]
    indices = indices.astype(int)-1
    data["pid"] = indices
    data["filename"] = filenames

    if optics is not None:
        og_names = set(optics["rlnOpticsGroup"].unique())
        for gn, g in data.groupby("rlnOpticsGroup", sort=False):
            if gn not in og_names:
                print(f"ERROR: optic group {gn} not available ({sorted(og_names)})")
                sys.exit(-1)
            ptcl_indices = g.index
            og_index = optics["rlnOpticsGroup"] == gn
            if "rlnPixelSize" in optics:
                data.loc[ptcl_indices, "apix"] = optics.loc[og_index, "rlnPixelSize"].astype(float).iloc[0]
    if "rlnPixelSize" in data:
        data.loc[:, "apix"] = data["rlnPixelSize"]
    if "rlnClassNumber" in data:
        data.loc[:, "class"] = data["rlnClassNumber"]
    if "rlnHelicalTubeID" in data:
        data.loc[:, "helicaltube"] = data["rlnHelicalTubeID"].astype(int)-1
    if "rlnAnglePsiPrior" in data:
        data.loc[:, "phi0"] = data["rlnAnglePsiPrior"].astype(float).round(3) - 90.0

    return data

In [ ]:
%%time
dataframe=star2dataframe(file_path)
dataframe_dup=dataframe

In [67]:
dataframe.columns

Index(['rlnAnglePsi', 'rlnAnglePsiFlipRatio', 'rlnAnglePsiPrior',
       'rlnAngleRot', 'rlnAngleTilt', 'rlnAngleTiltPrior', 'rlnClassNumber',
       'rlnCoordinateX', 'rlnCoordinateY', 'rlnCtfBfactor',
       'rlnCtfFigureOfMerit', 'rlnCtfMaxResolution', 'rlnCtfScalefactor',
       'rlnDefocusAngle', 'rlnDefocusU', 'rlnDefocusV', 'rlnGroupNumber',
       'rlnHelicalTrackLengthAngst', 'rlnHelicalTubeID', 'rlnImageName',
       'rlnLogLikeliContribution', 'rlnMaxValueProbDistribution',
       'rlnMicrographName', 'rlnNormCorrection', 'rlnNrOfSignificantSamples',
       'rlnOpticsGroup', 'rlnOriginXAngst', 'rlnOriginYAngst', 'rlnPhaseShift',
       'pid', 'filename', 'class', 'helicaltube', 'phi0'],
      dtype='object')

In [37]:
filament_data=dataframe.groupby(['filename','helicaltube'])

0 1.6526381174723306e-05 mins
1000 3.333489100138346e-05 mins
2000 3.333489100138346e-05 mins
3000 5.003213882446289e-05 mins
4000 6.655057271321614e-05 mins
5000 8.321603139241537e-05 mins
6000 9.989341100056967e-05 mins
7000 0.00011655489603678386 mins
8000 0.00013322830200195311 mins
9000 0.00014988183975219726 mins
10000 0.0001665631930033366 mins
11000 0.00018323659896850585 mins
12000 0.0001999020576477051 mins
13000 0.00021657943725585937 mins
14000 0.00023325284322102865 mins
15000 0.00024991830190022787 mins
16000 0.00026658773422241213 mins
17000 0.00028325716654459634 mins
18000 0.0002999265988667806 mins
19000 0.00031659603118896487 mins
20000 0.0003332654635111491 mins
21000 0.00034993489583333334 mins
22000 0.0003666083017985026 mins
23000 0.00038328170776367185 mins
24000 0.0003999511400858561 mins
25000 0.0004166245460510254 mins
26000 0.00043343305587768556 mins
27000 0.00045010248819986977 mins
28000 0.00046663681666056316 mins
29000 0.00048330227533976234 mins
30000 

In [47]:
%%time
start_time=time.time()
corpus=[]
dtype=[('class2D',int),('place',int),('index',int)]
for i in range(len(filament_index)):
    index=str(filament_index[i][0])+str(filament_index[i][0])
    df_select=dataframe[(dataframe['filename']==index[0])&(dataframe['helicaltube']==index[1])]
    filamnet_numpy=np.array([list(df_select['class']),list(df_select['pid']),list(df_select.index)]).T
    if i%1000==0:
        end_time=time.time()
        passed_time=(end_time-start_time)/60
        print(i,'%s mins' % passed_time)
    corpus.append(filamnet_numpy)

0 0.000588226318359375 mins
1000 2.355684590339661 mins
2000 4.665120955308279 mins


KeyboardInterrupt: 

In [45]:
filamnet_numpy=np.array([df_select['class'],df_select['pid'],df_select.index]).T

'Extract/job009/Tiff/EER/Images-Disc1/GridSquare_11149061/Data/FoilHole_11161627_Data_11149751_11149753_20210911_222712_EER.mrcs-0'

In [54]:
filament_index[0]

('Extract/job009/Tiff/EER/Images-Disc1/GridSquare_11149061/Data/FoilHole_11161627_Data_11149751_11149753_20210911_222712_EER.mrcs',
 0)

In [ ]:
def extarct_helical_select(self):
    data=self.data
    M = self.metadata.index('_rlnImageName')
    H = self.metadata.index('_rlnHelicalTubeID')
    C = self.metadata.index('_rlnClassNumber')
    print('finish reading')
    # extract helical parameters
    helicaldic = {}
    helicalnum = []
    count = -1
    dtype=[('class2D',int),('place',int),('index',int)]
    print('number of particles',len(data))
    for i, particle in enumerate(data):
        if i%10000==0:
            end_time=time.time()
            passed_time=(end_time-start_time)/60
            print(i,'%s mins' % passed_time)
        ID = particle[M][7:] + '-' + str(particle[H])
        if ID in helicalnum:
            n = str(helicalnum.index(ID))
            helicaldic[n]=helicaldic[n]+[(particle[C],particle[M][0:6],i)]
        else:
            helicalnum=helicalnum+[ID]
            n = str(helicalnum.index(ID))
            count += 1
            helicaldic[n] = [(particle[C],particle[M][0:6],i)]
    for i in range(len(helicaldic)):
        lst=np.array(helicaldic[str(i)],dtype=dtype)
        helicaldic[str(i)]=np.sort(lst,order='place')
    print('finish converting')
    #for i in range(5):
    #    print(helicaldic[str(i)])
    return helicaldic, helicalnum

In [79]:
%%time
start_time=time.time()
def extract_helical_select(dataframe):
    filament_index=list(filament_data.groups.keys())
    print('The filament number are: ',len(filament_index))
    helicaldic={}
    for i in range(len(filament_index)):
        name='-'.join(map(str, filament_index[i]))
        helicaldic[name]=[]
    for i in range(len(dataframe)):
        particle=dataframe.iloc[i]
        ID=str(particle['filename']) + '-' + str(particle['helicaltube'])
        helicaldic[ID]=helicaldic[ID]+[(particle['class'],particle['rlnImageName'][0:6],i)]
        if i%10000==0:
            end_time=time.time()
            passed_time=(end_time-start_time)/60
            print(i,'%s mins' % passed_time)
    return helicaldic, filament_index

0 0.0011834462483723959 mins
10000 0.023655410607655844 mins
20000 0.04596042235692342 mins
30000 0.06809873978296915 mins
40000 0.09053710699081421 mins
50000 0.11299215952555339 mins
60000 0.13543053468068442 mins
70000 0.15773555040359497 mins
80000 0.18000700076421103 mins
90000 0.2021286368370056 mins
100000 0.22441720962524414 mins
110000 0.24652212063471476 mins
120000 0.26847706238428753 mins
130000 0.2906154235204061 mins
140000 0.3129037698109945 mins
150000 0.33510876099268594 mins
160000 0.3575137972831726 mins
170000 0.3797187924385071 mins
180000 0.40160704453786217 mins
190000 0.42369534174601237 mins
200000 0.44591700633366904 mins
210000 0.4682887037595113 mins
220000 0.4905603726704915 mins
230000 0.5126320044199626 mins
240000 0.5347536404927572 mins
250000 0.5570253173510233 mins
260000 0.5788302103678385 mins
270000 0.6009182731310526 mins
280000 0.6230566183725993 mins
290000 0.6453618367513021 mins
300000 0.6674334645271301 mins
310000 0.6893717368443807 mins
320

In [80]:
helicaldic

{'Extract/job009/Tiff/EER/Images-Disc1/GridSquare_11149061/Data/FoilHole_11161627_Data_11149751_11149753_20210911_222712_EER.mrcs-0': [(65,
   '000007',
   59),
  (85, '000009', 60),
  (85, '000010', 61),
  (85, '000011', 62),
  (85, '000015', 63),
  (85, '000016', 64),
  (96, '000022', 65)],
 'Extract/job009/Tiff/EER/Images-Disc1/GridSquare_11149061/Data/FoilHole_11161627_Data_11149751_11149753_20210911_222712_EER.mrcs-1': [(14,
   '000024',
   36),
  (14, '000027', 37),
  (14, '000033', 38),
  (14, '000035', 39),
  (14, '000041', 40),
  (14, '000043', 41)],
 'Extract/job009/Tiff/EER/Images-Disc1/GridSquare_11149061/Data/FoilHole_11161627_Data_11149751_11149753_20210911_222712_EER.mrcs-2': [(35,
   '000050',
   47),
  (35, '000052', 48),
  (99, '000061', 49),
  (99, '000063', 50),
  (68, '000071', 51),
  (65, '000073', 52)],
 'Extract/job009/Tiff/EER/Images-Disc1/GridSquare_11149061/Data/FoilHole_11161627_Data_11149751_11149753_20210911_222712_EER.mrcs-3': [(62,
   '000086',
   0),
  